In [1]:
from pathlib import Path
import sys  

# Get my_package directory path from Notebook
parent_dir = str(Path().resolve().parents[0])
print(parent_dir)
# Add to sys.path

path_set = set(sys.path)
if parent_dir not in path_set:
    sys.path.insert(0, parent_dir)

print(sys.path)

# Comparator computation

This computes the Euclidean distance for each establishment from a base establishment of a consistent type (Academies, Maintained schools, SEN). Each establishment is given a weight against every other establishment and the top 60 for each establishment creates the comparator set for that establishment. This is repeated for all establishments in the establishment types set and then finally across all establishments across all establishment types.

In [2]:
import time 
import numpy as np
import pandas as pd
import src.pipeline.comparator_sets as comparators
import glob 
import os

start_time = time.time()
# Create and clean directory
from pathlib import Path
Path("output/comparator-sets").mkdir(parents=True, exist_ok=True)

# files = glob.glob("output/comparator-sets/*")
# for f in files:
#     os.remove(f)

# Prepare Academy and School Data

Here we prepare the academy and maintained school data by filling in missing values in NumberOfPupils, % Free School Meals and, % Sen with the mean (at this time). 

In [3]:
academy_data = comparators.prepare_data(pd.read_parquet("output/pre-processing/academies.parquet"))
ms_data = comparators.prepare_data(pd.read_parquet("output/pre-processing/maintained_schools.parquet"))
all_schools = comparators.prepare_data(pd.read_parquet("output/pre-processing/all_schools.parquet"))

In [4]:
all_schools[~all_schools.index.isna()]

# Compute the pupil and building comparators

This creates the comparators sets across both academy and maintained schools

In [5]:
academy_comparators = comparators.compute_comparator_set(academy_data)
# print('Academy done.')
# ms_comparators = comparators.compute_comparator_set(ms_data)
# print('MS done.')
# mixed_comparators = comparators.compute_comparator_set(all_schools)

In [6]:
# ms_comparators.to_parquet("output/comparator-sets/ms_comparators.parquet")
# academy_comparators.to_parquet("output/comparator-sets/academy_comparators.parquet")
# academy_comparators.to_parquet("output/comparator-sets/mixed_comparators.parquet")
# ms_data.to_parquet("output/comparator-sets/maintained_schools.parquet")
# academy_data.to_parquet("output/comparator-sets/academies.parquet")
# all_schools.to_parquet("output/comparator-sets/all_schools.parquet")

In [7]:
academy_comparators.head(5)


In [8]:
import pyodbc

pyodbc.drivers()

In [8]:
os.environ["DATABASE_CONNECTION_STRING"] = "Driver={ODBC Driver 18 for SQL Server};Server=localhost,1433;Database=Core;UID=sa;PWD=mystrong!Pa55word;Encrypt=no;TrustServerCertificate=yes;Connection Timeout=30"

import src.pipeline.database as db

db.insert_comparator_set("Default","Unmixed","2022", academy_comparators)